# Tabular example

In [ ]:
from fastai import *          # Quick accesss to most common functionality
from fastai.tabular import *  # Quick accesss to tabular functionality     # Access to example data provided with fastai

Tabular data should be in a Pandas `DataFrame`.

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [ ]:
dep_var = '>=50k'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [FillMissing, Categorify, Normalize]

In [ ]:
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cat_names=cat_names, cont_names=cont_names)

In [ ]:
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var)
                           .add_test(test, label=0)
                           .databunch())

In [ ]:
data.show_batch(rows=10)

workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
Private,Assoc-voc,Married-civ-spouse,Exec-managerial,Wife,White,False,0.6166,-0.8998,0.3599
Local-gov,12th,Widowed,Exec-managerial,Not-in-family,White,False,1.9360,-0.3397,-0.8135
Private,Assoc-voc,Married-civ-spouse,Exec-managerial,Husband,White,False,0.1036,-0.7128,0.3599
Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,False,0.4701,-0.8260,-0.4224
?,7th-8th,Married-civ-spouse,?,Husband,White,False,1.7161,-0.2803,-2.3781
Private,Some-college,Never-married,Transport-moving,Not-in-family,White,False,-0.4095,0.1358,-0.0312
Private,HS-grad,Never-married,Craft-repair,Unmarried,White,False,-0.2629,1.0632,-0.4224
Private,Bachelors,Widowed,Exec-managerial,Not-in-family,White,False,-0.9959,-0.3778,1.1422
Local-gov,Masters,Never-married,Prof-specialty,Unmarried,White,False,0.2502,0.0135,1.5334
Private,Some-college,Married-civ-spouse,Prof-specialty,Husband,White,False,0.5434,-0.2105,-0.0312


In [ ]:
learn = get_tabular_learner(data, layers=[200,100], metrics=accuracy)
learn.fit(1, 1e-2)

Total time: 00:04
epoch  train_loss  valid_loss  accuracy
1      0.373440    0.381403    0.825000  (00:04)



## Inference

In [ ]:
row = df.iloc[0]

In [ ]:
learn.predict(row)

(1, tensor(0), tensor([0.5647, 0.4353]))